# トレーニング用イメージ作成


In [ ]:
username = 'lc4ri099'

## awscliの準備

### インストール

In [ ]:
!sudo apt-get update && sudo apt-get upgrade -y && sudo apt-get install -y awscli

In [ ]:
!aws --version

### 設定

In [ ]:
target_region = 'ap-northeast-1'
!aws configure set default.region {target_region}
!aws configure set default.output json

認証情報としてAccessKeyIdとSecretAccessKeyを設定します。

コマンドに対する対話的な操作が必要ですから、JupyterのTerminal(treeページの[New] - [Terminal]から選択できます)から、 `aws configure` を実施してください。

`aws configure` の実施例:

```
$ aws configure
AWS Access Key ID [None]: (自身のアカウントのアクセスキー)
AWS Secret Access Key [None]: (自身のアカウントのシークレットアクセスキー)
Default region name [ap-northeast-1]: (Enter)
Default output format [json]: (Enter)
```

### boto3ライブラリの追加

ansibleからawsを触るために、ライブラリを追加しておきます。

In [ ]:
!conda install -y boto3 boto

## keypairの作成

クラウド上でLinuxマシンを起動する前に、ssh接続に必要となるkeypairを作成します。

In [ ]:
!aws ec2 delete-key-pair --key-name {username}
row = !aws ec2 create-key-pair --key-name {username} --query 'KeyMaterial' --output text

with open(username + '.pem', mode='w') as f:
    f.write('\n'.join(row))

import os
os.chmod(username + '.pem', 0o600)

#!aws ec2 delete-key-pair --key-name {username}

## Linuxマシンを起動する

ansibleでec2インスタンスを起動します。ディストリビューションはCentOS7とします。

In [ ]:
!aws --region ap-northeast-1 ec2 describe-images --owners aws-marketplace \
--filters Name=product-code,Values=aw0evgkw8e5c1q413zgy5pjce

In [ ]:
image = 'ami-045f38c93733dd48d' # CentOS7
instance_type = 't3.medium'
subnet = 'subnet-8a6618d1' # 先に作成済みのもの
sg = 'seminar-lc4ri' # 先に作成済みのもの

import json

row = !ansible localhost -m ec2 -a "region={target_region} key_name={username} \
profile=default wait=yes assign_public_ip=yes \
image={image} instance_type={instance_type} vpc_subnet_id={subnet} group={sg}"

while row[0].find('{') == -1:
    del row[0]
row[0] = "{"
linux = json.loads(''.join(row))
print(linux)

起動が完了するまで待ちます。

In [ ]:
!aws ec2 wait instance-status-ok --include-all-instances --instance-ids {linux['instance_ids'][0]}

起動したLinuxマシンでOSアップデートを実行します。

まず、接続情報を作成します。

In [ ]:
host = linux["instances"][0]["public_ip"]
ansible_user = "centos"
ansible_ssh_private_key_file = username + ".pem"

with open('hosts', mode='w') as f:
    f.write(host + ' ansible_user=' + ansible_user + ' ansible_ssh_private_key_file='
            + ansible_ssh_private_key_file)

作成した接続情報を使う前に、対象となるLinuxマシンを信頼できるホストとして、fingerprint登録しておきます。JupyterのTerminalを使って、ダミーのssh実行を行うと登録されます。

```
$ ssh a@（対象となるLinuxマシンのIPアドレス）
（中略）
Are you sure you want to continue connecting (yes/no)? yes←信頼しますよの意味で入力する
（以下略）
```

ssh接続を成功させたいわけではないので、接続自体は失敗させてしまって構いません。

実務では、ansibleの対象となるLinuxマシンが大量だったり不定だったりするかも知れません。その場合は、sshやansibleの設定で、fingerprint登録の有無を確認しないこともできます。

ansibleでyum updateを実行します。

In [ ]:
!ansible -i hosts all -a "yum update -y" --become

mariadbを入れて、httpdを入れて、php73を入れる準備。

In [ ]:
!ansible -i hosts all -a "yum install -y mariadb-server httpd-mmn epel-release" --become

In [ ]:
!ansible -i hosts all -a \
"rpm -Uvh http://rpms.famillecollet.com/enterprise/remi-release-7.rpm" --become

php73を入れる。

In [ ]:
!ansible -i hosts all -a "yum install -y --disablerepo=* \
--enablerepo=epel,remi,remi-safe,remi-php73 php php-mbstring php-mysqlnd" --become

httpdとmariadbを起動。httpdはenableしないで、トレーニング時に見つける対象にする。

In [ ]:
!ansible -i hosts all -a "systemctl start httpd" --become

In [ ]:
!ansible -i hosts all -a "systemctl start mariadb" --become

In [ ]:
!ansible -i hosts all -a "systemctl enable mariadb" --become

mariadbを起動したらセキュリティの初期化処理をterminalで実施。rootのパスワードは無しになってるので、そのままにしておく。他はデフォルト値でOK。

```
$ /usr/bin/mysql_secure_installation
```

mariadbにwp用のDBとユーザを作る。

In [ ]:
!ansible -i hosts all -a \
"mysql -u root -e 'create database wordpress default character set utf8;'" --become

In [ ]:
!ansible -i hosts all -a \
"mysql -u root -e \"grant all privileges on wordpress.* \
to user@localhost identified by 'password';\"" --become

wp-cliをインストール。

In [ ]:
!ansible -i hosts all -a \
"curl https://raw.githubusercontent.com/wp-cli/builds/gh-pages/phar/wp-cli.phar \
-o /usr/local/bin/wp" --become

In [ ]:
!ansible -i hosts all -a "chmod +x /usr/local/bin/wp" --become

In [ ]:
!ansible -i hosts all -a "wp --info"

wp-cliを使って、wpをインストール。

In [ ]:
!ansible -i hosts all -a "chmod 777 /var/www/html" --become

In [ ]:
!ansible -i hosts all -a "wp core download --locale=ja --path=/var/www/html"

In [ ]:
!ansible -i hosts all -a "wp core config --dbname=wordpress --dbuser=user --dbpass=password \
--dbhost=localhost --path=/var/www/html"

In [ ]:
!ansible -i hosts all -a "wp core install --url={host} --title=lc4ri \
--admin_name=admin --admin_password=admin --admin_email=admin@lc4ri.com  --path=/var/www/html"

In [ ]:
!ansible -i hosts all -a "wp config set WP_SITEURL http://{host} --path=/var/www/html"

In [ ]:
!ansible -i hosts all -a "wp config set WP_HOME http://{host} --path=/var/www/html"

hostにブラウザでアクセスするとwpを確認できる。admin/adminで入れる。

マシンイメージを取る。

In [ ]:
!aws ec2 create-image --instance-id {linux['instance_ids'][0]} \
--name "lc4ri-training" --description "wordpress is not running."

このイメージを起動すると、

- httpdが動いていない
- wp-config.phpのWP_SITEURLとWP_HOMEのIPアドレスが不正
- /var/www/html/wp-content/uploadsが書き込めないので写真をupできない

という不具合を確認できる。

## Linuxマシンを破棄する

In [ ]:
!aws ec2 terminate-instances --instance-ids {linux['instance_ids'][0]}